# Turing Machine as a Python Generator

In [2]:
import logging

from builtins import all
from itertools import islice, count

In [3]:
class TuringMachine:
    """Turing machine implementation.
    
    A machine is instantiated with a transitions; start, accept and
    reject states and a blank symbol. We assume that the input
    alphabet can be deducted from the transitions.

    :param dict transitions: a mapping from (state, symbol) tuples
    to (state, symbol, direction) tuple. Note that in theory δ is
    a transition *function*, but we expect a mapping, not a callable.
    Dirctions are either 'L' (for left) or 'R' (for right).
    
    :param start_state: the initial state of the machine.
    
    :param accpet_state: the accept state.
    
    :param reject_state: the reject state.
    
    :blank_symbol: the special symbold that marks the tape cell to be
    empty.
    
    We don't really care what the input alphabet Σ is. For a particular
    run of the machine it's the union of the input, symbols used in
    transitions and the blank symbol.
    
    The input on the tape is not part of a Turing machine, so it's not
    required on a Turing  machine instantiation. To execute a particular
    machine use the .run() instance method.
        
    """

    def __init__(self, transitions, start_state='q0', accept_state='qa', reject_state='qr', blank_symbol=''):
        self.blank_symbol = blank_symbol
        self.transitions = transitions
        self.start_state = start_state
        self.reject_state = reject_state
        
        self.states_to_actions = {
            accept_state: 'Accept',
            reject_state: 'Reject',
        }
        
    def run(self, input_):
        """Exectute the Turing machine for a particular input.
        
        :param input_: the input that is written on the tape.
        
        Given an input a machine can run forever or stop after a number
        of steps. So it would be great if we could write a function
        that potentially runs forever and it's up the the caller to
        decide how many steps are executed. Actually we should not even
        bother with this. On the other side, ^C is not the best way for
        a user to tell us to stop. Instead we give the user control to
        execute us one step at a time. This is what Python generators
        are (partially) for. The yield expression suspends us and gives
        controll to the caller until he or she decides to resume our
        execution. Have a look to [1] to get familliar with the yield
        keyord and generators, and hopefully never, ever write something
        like::
        
            result = []
            for i in range(len(other_items)):
                item = other_items[i]
                
                result.append(item * 3)
        
        At each step the method yield a (action, context) tuple.
        
        The action is either 'Accept', 'Reject' or None. 'Accept' and
        `Reject` are self explanatory and signal that the input is either
        accepted or rejected. None is returned in case the machine next to
        continue running.
        
        Context is a dictionary with the following keys:
        
        - 'state' the current state,
        - 'left_hand_side': the symbols on the left hand side of the current
           position.
        - 'symbol': the current symbol,
        - 'right_hand_side': the symbols on the righ hand side fo the current
           position.
        - 'direction': the direction the tape *has* moved. 'L' for left,
          'R' for right. None is yielded first time, since in the initial
           state there was no previous movement.
        
        
        [1] http://www.jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/
        
        """
        state = self.start_state

        # Theory doesn't say what the tape is, so we store the way
        # that is most suitable for us. We got two lists to store
        # symbols from left and righ hand sides from the current
        # symbol.
        #
        # Initially, there is the blank symbol on the left.
        # Note, that the element at 0 position is the *closest* to
        # the head.
        left_hand_side = [self.blank_symbol]
        if not input_:
            # In case input is empty, pretend that it consists
            # of a blank.
            input_ = [self.blank_symbol]
        # Consume the right most symbol of the input and make it
        # the current symbol, everything else is stored in a list
        # for righ side symbols.
        symbol, *right_hand_side = input_

        # We have not moved anywhere so far. We define it here, so
        # we can yield right in the beginning of the execution loop.
        direction = None
        
        while True:
            # Share with our state with the caller.
            # Also give them the chance to control our execution.
            action = self.states_to_actions.get(state)
            yield (
                action,
                {
                    'state': state,
                    'left_hand_side': left_hand_side,
                    'symbol': symbol,
                    'right_hand_side': right_hand_side,
                    'direction': direction,
                }
            )
            
            # Do the transition.
            state, symbol, direction = self.transitions.get(
                (state, symbol),
                (self.reject_state, symbol, 'L'),  # All other transitions are to the reject state.
            )

            # Check whether we need to stop executon.
            if action is not None:
                break

            # Shift to the left or to the right.
            # First we decide from what side we pop a symbol and to which append one.
            # This get rids of the duplicated code that actually pop and appends.
            if direction == 'L':
                to_append = right_hand_side
                to_pop = left_hand_side
            else:
                assert direction == 'R', 'L (left) and R (right) are the only corrected directions to move.'
                
                to_append = left_hand_side
                to_pop = right_hand_side
                
            to_append.insert(0, symbol)

            try:
                symbol = to_pop.pop(0)
            except IndexError:
                # In case we've reached the end of the tape,
                # pretend that we always have a blank symbol.
                symbol = self.blank_symbol
                    
    def accepts(self, input_, step_limit=100):
        *_, (action, _) = islice(self.run(input_), step_limit)

        if action is not None:
            return action == 'Accept'
        else:
            logging.warn('The step limit of %s steps  is reached!', step_limit)
        
    def rejects(self, input_, **kwargs):
        accepts = self.accepts(input_, **kwargs)
        
        if accepts is not None:
            return not accepts
        
    def debug(self, input_, step_limit=None):
        for action, context in islice(self.run(input_), step_limit):
            print(
                '{state:<30} {left}{b}{symbol}{f}{right}'.format(
                    left=''.join(context['left_hand_side']),
                    right=''.join(context['right_hand_side']),
                    b='\x1b[47;1m',
                    f='\x1b[0m',
                    **context
                )
            )

# One hash

In [4]:
one_hash = TuringMachine(
    {
        ('q0', '#'): ('saw_#', '#', 'R'),
        ('saw_#', ''): ('qa', '', 'R'),
    }
)

In [5]:
execution = one_hash.run('#')

In [6]:
next(execution)

(None,
 {'direction': None,
  'left_hand_side': [''],
  'right_hand_side': [],
  'state': 'q0',
  'symbol': '#'})

In [7]:
next(execution)

(None,
 {'direction': 'R',
  'left_hand_side': ['#', ''],
  'right_hand_side': [],
  'state': 'saw_#',
  'symbol': ''})

In [8]:
next(execution)

('Accept',
 {'direction': 'R',
  'left_hand_side': ['', '#', ''],
  'right_hand_side': [],
  'state': 'qa',
  'symbol': ''})

In [9]:
one_hash.accepts('#')

True

In [10]:
assert one_hash.accepts('#')

In [11]:
assert one_hash.rejects('##')

In [12]:
assert one_hash.rejects('')

In [13]:
assert one_hash.rejects(' ')

In [14]:
assert one_hash.accepts('#', step_limit=1) is None

# Two hashes

In [15]:
two_hashes = TuringMachine(
    {
        ('q0', '#'): ('saw_#', '#', 'R'),
        ('saw_#', '#'): ('saw_##', '#', 'R'),
        ('saw_##', ''): ('qa', '', 'R'),
    }
)

In [16]:
assert two_hashes.accepts('##')

In [17]:
assert two_hashes.rejects('#')

In [18]:
assert two_hashes.rejects('###')

# Two same words separated by # (w#w)

In [19]:
w_hash_w = TuringMachine(
    {
        ('q0', '#'): ('End', '#', 'R'),
        ('End', ''): ('qa', '', 'R'),

        ('q0', '0'): ('FindDelimiter0', 'X', 'R'),
        ('FindDelimiter0', '#'): ('Check0', '#', 'R'),
        ('Check0', '0'): ('FindLeftmost', 'X', 'L'),

        ('q0', '1'): ('FindDelimiter1', 'X', 'R'),
        ('FindDelimiter1', '#'): ('Check1', '#', 'R'),
        ('Check1', '1'): ('FindLeftmost', 'X', 'L'),

        ('FindLeftmost', '0'): ('FindLeftmost', '0', 'L'),
        ('FindLeftmost', '1'): ('FindLeftmost', '1', 'L'),
        ('FindLeftmost', 'X'): ('FindLeftmost', 'X', 'L'),
        ('FindLeftmost', '#'): ('FindLeftmost', '#', 'L'),
        ('FindLeftmost', ''): ('FindNext', '', 'R'),
        
        ('FindNext', 'X'): ('FindNext', 'X', 'R'),
        ('FindNext', '0'): ('FindDelimiter0', 'X', 'R'),
        ('FindNext', '1'): ('FindDelimiter1', 'X', 'R'),
        ('FindNext', '#'): ('End', '#', 'R'),
        
        ('FindDelimiter0', '0'): ('FindDelimiter0', '0', 'R'),
        ('FindDelimiter0', '1'): ('FindDelimiter0', '1', 'R'),
        ('FindDelimiter1', '0'): ('FindDelimiter1', '0', 'R'),
        ('FindDelimiter1', '1'): ('FindDelimiter1', '1', 'R'),
        
        ('Check0', 'X'): ('Check0', 'X', 'R'),
        ('Check1', 'X'): ('Check1', 'X', 'R'),
        
        ('End', 'X'): ('End', 'X', 'R')
    }
)

In [20]:
assert w_hash_w.accepts('#')

In [21]:
assert w_hash_w.accepts('0#0')

In [22]:
w_hash_w.debug('0#0')

q0                             0#0
FindDelimiter0                 X#0
Check0                         #X0
FindLeftmost                   X#X
FindLeftmost                   X#X
FindLeftmost                   X#X
FindNext                       X#X
FindNext                       X#X
End                            #XX
End                            X#X
qa                             X#X


In [23]:
assert w_hash_w.accepts('1#1')

In [24]:
assert w_hash_w.accepts('0000000000000#0000000000000', step_limit=10000)

In [25]:
assert w_hash_w.accepts('1001#1001')

In [26]:
assert w_hash_w.rejects('10#1')

In [27]:
assert w_hash_w.rejects('1#01')

In [28]:
assert w_hash_w.rejects('1#1#')

In [29]:
assert w_hash_w.rejects('1##1')

In [30]:
w_hash_w.debug('101#1010')

q0                             101#1010
FindDelimiter1                 X01#1010
FindDelimiter1                 0X1#1010
FindDelimiter1                 10X#1010
Check1                         #10X1010
FindLeftmost                   10X#X010
FindLeftmost                   0X1#X010
FindLeftmost                   X01#X010
FindLeftmost                   X01#X010
FindLeftmost                   X01#X010
FindNext                       X01#X010
FindNext                       X01#X010
FindDelimiter0                 XX1#X010
FindDelimiter0                 1XX#X010
Check0                         #1XXX010
Check0                         X#1XX010
FindLeftmost                   #1XXXX10
FindLeftmost                   1XX#XX10
FindLeftmost                   XX1#XX10
FindLeftmost                   XX1#XX10
FindLeftmost                   XX1#XX10
FindLeftmost                   XX1#XX10
FindNext                       XX1#XX10
FindNext                       XX1#XX10
FindNext                       XX1#XX10


# Check for longer words

In [31]:
def generate_words():
    for n in count():
        yield bin(n)[2:]

In [32]:
assert all(w_hash_w.accepts('{0}#{0}'.format(w), step_limit=1000) for w in islice(generate_words(), 1000))

# Replace processed symbols on the left of # with blanks.

In [37]:
w_hash_w_blank = TuringMachine(
    {
        ('q0', '#'): ('End', '#', 'R'),
        ('End', ''): ('qa', '', 'R'),

        ('q0', '0'): ('FindDelimiter0', '', 'R'),
        ('FindDelimiter0', '#'): ('Check0', '#', 'R'),
        ('Check0', '0'): ('FindLeftmost', 'X', 'L'),

        ('q0', '1'): ('FindDelimiter1', '', 'R'),
        ('FindDelimiter1', '#'): ('Check1', '#', 'R'),
        ('Check1', '1'): ('FindLeftmost', 'X', 'L'),

        ('FindLeftmost', '0'): ('FindLeftmost', '0', 'L'),
        ('FindLeftmost', '1'): ('FindLeftmost', '1', 'L'),
        ('FindLeftmost', 'X'): ('FindLeftmost', 'X', 'L'),
        ('FindLeftmost', '#'): ('FindLeftmost', '#', 'L'),
        ('FindLeftmost', ''): ('FindNext', '', 'R'),
        
        ('FindNext', '0'): ('FindDelimiter0', '', 'R'),
        ('FindNext', '1'): ('FindDelimiter1', '', 'R'),
        ('FindNext', '#'): ('End', '#', 'R'),
        
        ('FindDelimiter0', '0'): ('FindDelimiter0', '0', 'R'),
        ('FindDelimiter0', '1'): ('FindDelimiter0', '1', 'R'),
        ('FindDelimiter1', '0'): ('FindDelimiter1', '0', 'R'),
        ('FindDelimiter1', '1'): ('FindDelimiter1', '1', 'R'),
        
        ('Check0', 'X'): ('Check0', 'X', 'R'),
        ('Check1', 'X'): ('Check1', 'X', 'R'),
        
        ('End', 'X'): ('End', 'X', 'R')
    }
)

In [38]:
w_hash_w_blank.debug('01#01')

q0                             01#01
FindDelimiter0                 1#01
FindDelimiter0                 1#01
Check0                         #101
FindLeftmost                   1#X1
FindLeftmost                   1#X1
FindLeftmost                   1#X1
FindNext                       1#X1
FindDelimiter1                 #X1
Check1                         #X1
Check1                         X#1
FindLeftmost                   #XX
FindLeftmost                   #XX
FindLeftmost                   #XX
FindNext                       #XX
End                            #XX
End                            X#X
End                            XX#
qa                             XX#


In [39]:
len(list(w_hash_w_blank.run('0110#0110')))

51

In [40]:
len(list(w_hash_w.run('0110#0110')))

71

In [41]:
from IPython.display import HTML

HTML(
    """
    <div id="disqus_thread"></div>
    <script type="text/javascript">
        /* * * CONFIGURATION VARIABLES: EDIT BEFORE PASTING INTO YOUR WEBPAGE * * */
        var disqus_shortname = 'notebookcomments'; // required: replace example with your forum shortname

        /* * * DON'T EDIT BELOW THIS LINE * * */
        (function() {
            var dsq = document.createElement('script'); dsq.type = 'text/javascript'; dsq.async = true;
            dsq.src = '//' + disqus_shortname + '.disqus.com/embed.js';
            (document.getElementsByTagName('head')[0] || document.getElementsByTagName('body')[0]).appendChild(dsq);
        })();
    </script>
    <noscript>Please enable JavaScript to view the <a href="http://disqus.com/?ref_noscript">comments powered by Disqus.</a></noscript>
    <a href="http://disqus.com" class="dsq-brlink">comments powered by <span class="logo-disqus">Disqus</span></a>
    """
)

